In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vi-data/examples (1).json
/kaggle/input/vi_model/pytorch/default/1/model/adapter_model.safetensors
/kaggle/input/vi_model/pytorch/default/1/model/adapter_config.json
/kaggle/input/vi_model/pytorch/default/1/model/README.md
/kaggle/input/vi_model/pytorch/default/1/model/tokenizer.json
/kaggle/input/vi_model/pytorch/default/1/model/tokenizer_config.json
/kaggle/input/vi_model/pytorch/default/1/model/special_tokens_map.json
/kaggle/input/vina-llama7b/pytorch/default/1/model/adapter_model.safetensors
/kaggle/input/vina-llama7b/pytorch/default/1/model/adapter_config.json
/kaggle/input/vina-llama7b/pytorch/default/1/model/README.md
/kaggle/input/vina-llama7b/pytorch/default/1/model/tokenizer.json
/kaggle/input/vina-llama7b/pytorch/default/1/model/tokenizer_config.json
/kaggle/input/vina-llama7b/pytorch/default/1/model/special_tokens_map.json


In [4]:
 !pip install fastapi pydantic transformers peft nest_asyncio pyngrok uvicorn SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00


In [5]:


from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Khởi động FastAPI
app = FastAPI()

# Đường dẫn tới thư mục chứa mô hình trên Kaggle
model_path = "/kaggle/input/vi_model/pytorch/default/1/model"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

# Load Base Model
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto", 
    torch_dtype=torch.float16
)

# Load cấu hình của LoRA Adapter
peft_config = PeftConfig.from_pretrained(model_path)

# Load LoRA Adapter
model = PeftModel.from_pretrained(
    base_model, 
    model_path, 
    is_trainable=True, 
    config=peft_config
)

print("LoRA Adapter loaded successfully!")




config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

LoRA Adapter loaded successfully!


In [6]:
import re

def generate_query(question):
    input_text = f"### Instruction:\nTạo truy vấn Logical Form để lấy thông tin tương ứng với câu hỏi đã cho. \n\n### Input:\nQuestion: {question}\n\n### Output:\n"
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    
    output_ids = model.generate(
        input_ids, 
        max_new_tokens=100,
        num_beams=5,              
        num_beam_groups=1,        
        num_return_sequences=5,    
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )
    
    def clean_output(text):
        text = text.split("Output:")[-1].strip()  
        text = re.split(r"\n+", text)[0]  
        return text

    output_texts = [clean_output(tokenizer.decode(out, skip_special_tokens=True)) for out in output_ids]
    
    return output_texts  


In [7]:
import re

def convert_s_expression_to_sparql_1(s_expression):
    pattern = r"\( ASK \( AND \( (\w+) (\w+) (\w+) \) \( \1 \2 (\w+) \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        entity, predicate, obj1, obj2 = match.groups()
        return f"ASK WHERE {{ wd:{entity} wdt:{predicate} wd:{obj1} . wd:{entity} wdt:{predicate} wd:{obj2} }}"
    
    return None
def convert_s_expression_to_sparql_2(s_expression):
    pattern = r"\( MAX \( JOIN \( R (\w+) \) \( JOIN (\w+) (\w+) \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel2, rel1, type_entity = match.groups()
        return f"SELECT ?ent WHERE {{ ?ent wdt:{rel1} wd:{type_entity} . ?ent wdt:{rel2} ?obj }} ORDER BY DESC(?obj) LIMIT 5"
    
    return None
def convert_s_expression_to_sparql_3(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( JOIN \( R (\w+) \) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel2, rel1, entity = match.groups()
        return f"SELECT ?answer WHERE {{ wd:{entity} wdt:{rel1} ?X . ?X wdt:{rel2} ?answer }}"
    
    return None
def convert_s_expression_to_sparql_4(s_expression):
    pattern = r"\( AND \( JOIN \( R (\w+) \) (\w+) \) \( JOIN (\w+) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel1, en1, rel2, en2 = match.groups()
        return f"SELECT DISTINCT ?obj WHERE {{ wd:{en1} wdt:{rel1} ?obj . ?obj wdt:{rel2} wd:{en2} }}"
    
    return None
def convert_s_expression_to_sparql_5(s_expression):
    pattern = r"\( MIN \( JOIN \( R (\w+) \) \( AND \( JOIN (\w+) (\w+) \) \( JOIN (\w+) (\w+) \) \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel, rel1, obj1, rel2, obj2 = match.groups()
        return f"SELECT ?ent WHERE {{ ?ent wdt:{rel1} wd:{obj1} . ?ent wdt:{rel} ?obj . ?ent wdt:{rel2} wd:{obj2} }} ORDER BY ASC(?obj) LIMIT 5"
    
    return None
def convert_s_expression_to_sparql_6(s_expression):
    pattern = r"\( MAX \( JOIN \( R (\w+) \) \( AND \( JOIN (\w+) (\w+) \) \( JOIN (\w+) (\w+) \) \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel, rel1, obj1, rel2, obj2 = match.groups()
        return f"SELECT ?ent WHERE {{ ?ent wdt:{rel1} wd:{obj1} . ?ent wdt:{rel} ?obj . ?ent wdt:{rel2} wd:{obj2} }} ORDER BY DESC(?obj) LIMIT 5"
    
    return None
def convert_s_expression_to_sparql_7(s_expression):
    pattern = r"\( JOIN (\w+) (\w+) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        predicate, obj = match.groups()
        return f"SELECT DISTINCT ?answer WHERE {{ ?answer wdt:{predicate} wd:{obj} }}"
    
    return None

def convert_s_expression_to_sparql_8(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( AND \( JOIN \( R \1 \) (\w+) \) \( JOIN (\w+) (\w+) \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        pred, entity, qualifier, qual_obj = match.groups()
        return f"SELECT ?obj WHERE {{ wd:{entity} p:{pred} ?s . ?s ps:{pred} ?obj . ?s pq:{qualifier} wd:{qual_obj} }}"
    
    return None

def convert_s_expression_to_sparql_9(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( AND \( JOIN \( R (\w+) \) (\w+) \) \( JOIN \2 (\w+) \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        qualifier, pred, entity, obj = match.groups()
        return f"SELECT ?value WHERE {{ wd:{entity} p:{pred} ?s . ?s ps:{pred} wd:{obj} . ?s pq:{qualifier} ?value }}"
    
    return None
def convert_s_expression_to_sparql_10(s_expression):
    pattern = r"\( AND \( JOIN (\w+) (\w+) \) \( JOIN (\w+) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        pred, obj, p2, type_obj = match.groups()
        return f"SELECT DISTINCT ?sbj WHERE {{ ?sbj wdt:{pred} wd:{obj} . ?sbj wdt:{p2} wd:{type_obj} }}"
    
    return None

def convert_s_expression_to_sparql_11(s_expression):
    pattern = r"\( AND \( JOIN \( R (\w+) \) (\w+) \) \( JOIN (\w+) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel1, entity, rel2, obj = match.groups()
        return f"SELECT ?answer WHERE {{ wd:{entity} wdt:{rel1} ?answer . ?answer wdt:{rel2} wd:{obj} }}"
    
    return None

def convert_s_expression_to_sparql_12(s_expression):
    pattern = r"\( ASK \( (\=|>=|<=|>|<) \( JOIN \( R (\w+) \) (\w+) \) (\d+(\.\d+)?) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        operator, predicate, entity, value, _ = match.groups()
        return f"ASK WHERE {{ wd:{entity} wdt:{predicate} ?obj filter(?obj {operator} {value}) }}"
    
    return None

def convert_s_expression_to_sparql_13(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) (\w+) \) \( JOIN \( R (\w+) \) \2 \)"
    match = re.match(pattern, s_expression)
    
    if match:
        pred1, entity, pred2 = match.groups()
        return f"SELECT ?ans_1 ?ans_2 WHERE {{ wd:{entity} wdt:{pred1} ?ans_1 . wd:{entity} wdt:{pred2} ?ans_2 }}"
    
    return None

def convert_s_expression_to_sparql_14(s_expression):
    pattern = r"\( COUNT \( JOIN (\w+) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        predicate, obj = match.groups()
        return f"SELECT (COUNT(?sub) AS ?value ) {{ ?sub wdt:{predicate} wd:{obj} }}"
    
    return None

def convert_s_expression_to_sparql_15(s_expression):
    pattern = r"\( COUNT \( JOIN \( R (\w+) \) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        predicate, entity = match.groups()
        return f"SELECT (COUNT(?obj) AS ?value ) {{ wd:{entity} wdt:{predicate} ?obj }}"
    
    return None

def convert_s_expression_to_sparql_16(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) (\w+) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        predicate, entity = match.groups()
        return f"SELECT DISTINCT ?answer WHERE {{ wd:{entity} wdt:{predicate} ?answer }}"
    
    return None

def convert_s_expression_to_sparql_17(s_expression):
    pattern = r"\( CHAR \( AND \( JOIN (\w+) (\w+) \) \( JOIN (\w+) (\w+) \) \) ['\"](.*?)['\"] \)"
    match = re.match(pattern, s_expression)
    
    if match:
        pred1, obj1, pred2, obj2, char = match.groups()
        return f"""SELECT DISTINCT ?sbj ?sbj_label WHERE {{ 
            ?sbj wdt:{pred1} wd:{obj1} . 
            ?sbj wdt:{pred2} wd:{obj2} . 
            ?sbj rdfs:label ?sbj_label . 
            FILTER(STRSTARTS(lcase(?sbj_label), '{char}')) . 
            FILTER (lang(?sbj_label) = 'en') 
        }} LIMIT 25"""
    
    return None

def convert_s_expression_to_sparql_18(s_expression):
    pattern = r"\( ASK \( (\w+) (\w+) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        entity, predicate, obj = match.groups()
        return f"ASK WHERE {{ wd:{entity} wdt:{predicate} wd:{obj} }}"
    
    return None

def convert_s_expression_to_sparql_19(s_expression):
    pattern = r"\( WORD \( JOIN (\w+) (\w+) \) ['\"](.*?)['\"] \)"
    match = re.match(pattern, s_expression)
    
    if match:
        predicate, obj, word = match.groups()
        return f"""SELECT DISTINCT ?sbj ?sbj_label WHERE {{ 
            ?sbj wdt:{predicate} wd:{obj} . 
            ?sbj rdfs:label ?sbj_label . 
            FILTER(CONTAINS(lcase(?sbj_label), '{word}')) . 
            FILTER (lang(?sbj_label) = 'en') 
        }} LIMIT 25"""
    
    return None

def convert_s_expression_to_sparql_20(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( JOIN \( R (\w+) \) (\w+) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel2, rel1, entity = match.groups()
        return f"SELECT ?answer WHERE {{ wd:{entity} wdt:{rel1} ?X . ?X wdt:{rel2} ?answer }}"
    
    return None

def convert_s_expression_to_sparql_21(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( FILTER \( JOIN \( R (\w+) \) \( JOIN \( R (\w+) \) (\w+) \) \) \( ['\"](.*?)['\"] \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        pred1, pred2, pred3, entity, value = match.groups()
        return f"""SELECT ?obj WHERE {{ 
            wd:{entity} p:{pred1} ?s . 
            ?s ps:{pred1} ?obj . 
            ?s pq:{pred2} ?x filter(contains(?x,'{value}')) 
        }}"""
    
    return None

def convert_s_expression_to_sparql_22(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( FILTER \( JOIN \( R (\w+) \) (\w+) \) \( ['\"](.*?)['\"] \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel1, rel2, entity, value = match.groups()
        return f"""SELECT ?answer WHERE {{ 
            wd:{entity} wdt:{rel1} ?answer . 
            ?answer wdt:{rel2} ?x FILTER(contains(?x,'{value}')) 
        }}"""
    
    return None

def convert_s_expression_to_sparql_23(s_expression):
    pattern = r"\( JOIN \( R (\w+) \) \( FILTER \( JOIN \( R (\w+) \) \( JOIN \( R (\w+) \) (\w+) \) \) \( ['\"](.*?)['\"] \) \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        pred1, pred2, pred3, entity, value = match.groups()
        return f"""SELECT ?obj WHERE {{ 
            wd:{entity} p:{pred3} ?s . 
            ?s ps:{pred3} ?obj . 
            ?s pq:{pred2} ?x filter(contains(YEAR(?x),'{value}')) 
        }}"""
    
    return None

def convert_s_expression_to_sparql_24(s_expression):
    pattern = r"\( FILTER \( JOIN \( R (\w+) \) \( JOIN \( R (\w+) \) (\w+) \) \) \( YEAR ['\"](.*?)['\"] \) \)"
    match = re.match(pattern, s_expression)
    
    if match:
        rel2, rel1, entity, value = match.groups()
        return f"""SELECT ?answer WHERE {{ 
            wd:{entity} wdt:{rel1} ?answer . 
            ?answer wdt:{rel2} ?x FILTER(contains(YEAR(?x),'{value}')) 
        }}"""
    
    return None


def convert_s_expression_to_sparql_25(s_expression):
    pattern = r"\( WORD \( JOIN (\w+) (\w+) \) ['\"](.*?)['\"] \)"
    match = re.match(pattern, s_expression)
    
    if match:
        predicate, obj, word = match.groups()
        return f"""SELECT DISTINCT ?sbj ?sbj_label WHERE {{ 
            ?sbj wdt:{predicate} wd:{obj} . 
            ?sbj rdfs:label ?sbj_label . 
            FILTER(STRSTARTS(lcase(?sbj_label), '{word}')) . 
            FILTER (lang(?sbj_label) = 'en') 
        }} LIMIT 25"""
    
    return None

conversion_functions = [
    convert_s_expression_to_sparql_1, convert_s_expression_to_sparql_2, convert_s_expression_to_sparql_3,
    convert_s_expression_to_sparql_4, convert_s_expression_to_sparql_5, convert_s_expression_to_sparql_6,
    convert_s_expression_to_sparql_7, convert_s_expression_to_sparql_8, convert_s_expression_to_sparql_9,
    convert_s_expression_to_sparql_10, convert_s_expression_to_sparql_11, convert_s_expression_to_sparql_12,
    convert_s_expression_to_sparql_13, convert_s_expression_to_sparql_14, convert_s_expression_to_sparql_15,
    convert_s_expression_to_sparql_16, convert_s_expression_to_sparql_17, convert_s_expression_to_sparql_18,
    convert_s_expression_to_sparql_19, convert_s_expression_to_sparql_20, convert_s_expression_to_sparql_21,
    convert_s_expression_to_sparql_22, convert_s_expression_to_sparql_23, convert_s_expression_to_sparql_24,
    convert_s_expression_to_sparql_25
]

def convert_s_expression_to_sparql(s_expression):
    """
    Chuyển đổi S-expression sang SPARQL bằng cách thử từng template phù hợp.
    """
    for func in conversion_functions:
        sparql_query = func(s_expression)
        if sparql_query:
            return sparql_query  # Trả về SPARQL nếu có kết quả hợp lệ

    return "UNKNOWN"  # Trả về UNKNOWN nếu không khớp với bất kỳ mẫu nào


In [8]:
from itertools import product
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import re
import requests
from sklearn.preprocessing import normalize

import time

def is_valid_expression(expr):
    """Kiểm tra tính hợp lệ của biểu thức bằng cách đếm số ngoặc mở và đóng."""
    count = 0
    for char in expr:
        if char == '(':
            count += 1
        elif char == ')':
            count -= 1
        if count < 0:
            return False  # Gặp ngoặc đóng trước ngoặc mở
    return count == 0

def search_wikidata_entities(label: str, language: str = "vi"):
    """
    Tìm các thực thể Wikidata có mã Q cụ thể liên quan đến nhãn.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "type": "item",  # Chỉ tìm thực thể (bỏ qua quan hệ P)
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Lỗi truy vấn Wikidata: {e}")
        return []

    if "search" in data:
        return [item["id"] for item in data["search"]][:9]  # Lấy mã thực thể Q

    return []

def search_wikidata_relations(label: str, language: str = "vi"):
    """
    Tìm các mã quan hệ (P...) trên Wikidata dựa vào nhãn.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "type": "property",  # Chỉ tìm quan hệ (P...)
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Lỗi truy vấn Wikidata: {e}")
        return []

    if "search" in data:
        return [item["id"] for item in data["search"]][:9]  # Lấy mã quan hệ P

    return []


def fix_unbalanced_parentheses(expr):
    """Loại bỏ ngoặc đóng dư nếu có."""
    while not is_valid_expression(expr) and expr.endswith(')'):
        expr = expr[:-1]
    return expr


def parse_nsexpr(expr):
    """
    Chuyển chuỗi biểu thức thành cây cấu trúc dạng nested list.
    Hàm này dùng duyệt ký tự, khi gặp '(' sẽ tìm phần con cho đến khi khớp với ')',
    và giữ nguyên nội dung trong ngoặc vuông.
    """
    tokens = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            i += 1
        elif expr[i] == '(':
            # Tìm phần con của biểu thức trong ngoặc đơn
            count = 1
            j = i + 1
            while j < len(expr) and count > 0:
                if expr[j] == '(':
                    count += 1
                elif expr[j] == ')':
                    count -= 1
                j += 1
            # Đệ quy phân tích phần con (loại bỏ ngoặc bao ngoài)
            subtree = parse_nsexpr(expr[i+1:j-1])
            tokens.append(subtree)
            i = j
        elif expr[i] == '[':
            # Giữ nguyên nội dung trong ngoặc vuông
            j = expr.find(']', i)
            if j == -1:
                return ""
                raise ValueError("Không tìm thấy dấu ']' kết thúc.")
                
            token = expr[i:j+1].strip()
            tokens.append(token)
            i = j + 1
        else:
            # Đọc một token cho đến khi gặp khoảng trắng hoặc ngoặc
            j = i
            while j < len(expr) and (not expr[j].isspace()) and expr[j] not in ['(', ')']:
                j += 1
            tokens.append(expr[i:j])
            i = j
    return tokens

def collect_labels(tree):
    """
    Duyệt cây cấu trúc (nested list) để thu thập các nhãn của quan hệ và thực thể.
    Giả sử:
      - Biểu thức JOIN có dạng: ["JOIN", relation_part, entity_part]
      - Phần relation_part: nếu là list và bắt đầu bằng "R", thì phần thứ hai chứa nhãn quan hệ (dạng "[ label ]"). 
        Nếu là chuỗi dạng "[ label ]" thì đó cũng là nhãn quan hệ.
      - Phần entity_part: nếu là chuỗi dạng "[ label ]" thì đó là nhãn thực thể, nếu là list thì xử lý đệ quy.
      - Biểu thức AND sẽ có nhiều biểu thức con.
    """
    relations = []
    entities = []
    
    if isinstance(tree, list) and tree:
        # Nếu token đầu tiên là JOIN hoặc AND
        op = tree[0]
        if isinstance(op, str):
            op_upper = op.upper()
        else:
            op_upper = ""
        
        if op_upper == "JOIN":
            # Xử lý phần quan hệ
            if len(tree) >= 2:
                rel_part = tree[1]
                # Nếu là list dạng [ "R", "[ label ]" ]
                if isinstance(rel_part, list) and len(rel_part) >= 2 and isinstance(rel_part[0], str) and rel_part[0].upper() == "R":
                    token = rel_part[1]
                    if isinstance(token, str) and token.startswith('[') and token.endswith(']'):
                        rel_label = token[1:-1].strip()
                        relations.append(rel_label)
                # Nếu là chuỗi dạng "[ label ]"
                elif isinstance(rel_part, str) and rel_part.startswith('[') and rel_part.endswith(']'):
                    rel_label = rel_part[1:-1].strip()
                    relations.append(rel_label)
                else:
                    # Nếu không đúng định dạng, duyệt đệ quy
                    sub_rel, sub_ent = collect_labels(rel_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
            # Xử lý phần thực thể
            if len(tree) >= 3:
                ent_part = tree[2]
                if isinstance(ent_part, list):
                    sub_rel, sub_ent = collect_labels(ent_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
                elif isinstance(ent_part, str) and ent_part.startswith('[') and ent_part.endswith(']'):
                    ent_label = ent_part[1:-1].strip()
                    entities.append(ent_label)
        elif op_upper == "AND":
            # Với AND, duyệt tất cả các phần con
            for sub in tree[1:]:
                sub_rel, sub_ent = collect_labels(sub)
                relations.extend(sub_rel)
                entities.extend(sub_ent)
        else:
            # Nếu không phải JOIN hay AND, duyệt tất cả các phần tử nếu chúng là list
            for elem in tree:
                if isinstance(elem, list):
                    sub_rel, sub_ent = collect_labels(elem)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
    return relations, entities



def extract_entities_and_relations(normed_expr):

    if not normed_expr or len(normed_expr) == 0:  # Kiểm tra nếu normed_expr rỗng
        return [], []
    
    if normed_expr[0] != "(":
        return [], []
    
    tree = parse_nsexpr(normed_expr)
    if tree is None:
        return [], []  # Trả về danh sách rỗng nếu parse thất bại
    
    return collect_labels(tree)


class SExpressionParser:
    def __init__(self):
        self.var_counter = 1  # Đếm số biến trung gian (?X1, ?X2, ...)

    def get_new_var(self):
        """Tạo biến trung gian mới."""
        var_name = f"?X{self.var_counter}"
        self.var_counter += 1
        return var_name

    def parse_s_expr(self, s_expr):
        """Chuyển đổi S-Expression thành danh sách lồng nhau."""
        s_expr = re.sub(r'\(', ' ( ', s_expr)
        s_expr = re.sub(r'\)', ' ) ', s_expr)
        tokens = s_expr.split()
        return self.build_tree(tokens)

    def build_tree(self, tokens):
        """Chuyển đổi danh sách token thành cây lồng nhau."""
        if not tokens:
            return None
        token = tokens.pop(0)
        if token == "(":
            sub_expr = []
            while tokens[0] != ")":
                sub_expr.append(self.build_tree(tokens))
            tokens.pop(0)  # Bỏ dấu ")"
            return sub_expr
        elif token == ")":
            raise ValueError("Unexpected ')'")
        else:
            return token

    def process_join(self, expr, target_var):
        """
        Xử lý JOIN, tạo triple SPARQL.
        """
        triples = []
        if not isinstance(expr, list):
            return expr, triples

        if expr[0] == "AND":
            # Xử lý từng JOIN trong AND riêng lẻ
            for sub_expr in expr[1:]:
                _, sub_triples = self.process_join(sub_expr, target_var)
                triples.extend(sub_triples)
            return target_var, triples

        if expr[0] == "JOIN":
            right_expr = expr[2]
            right_triples = []
            if isinstance(right_expr, list) and right_expr[0] == "JOIN":
                right_var, right_triples = self.process_join(right_expr, self.get_new_var())
            else: 
                right_var = right_expr
            
            # Xử lý nhánh trái
            left_expr = expr[1]
            if isinstance(left_expr, list) and left_expr[0] == "R":
                rel = left_expr[1]
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([right, f"wdt:{rel}", left])
            else:
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([left, f"wdt:{left_expr}", right])

            # Thêm các triples từ nhánh phải trước khi thêm triple chính
            triples = right_triples + triples
            return target_var, triples

        return expr, triples

    def s_expr_to_sparql(self, s_expr):
        """Chuyển đổi từ S-Expression sang SPARQL."""
        if s_expr.count("(") != s_expr.count(")"):
            return None
        if s_expr.count("[") != s_expr.count("]"):
            return None
        parsed_expr = self.parse_s_expr(s_expr)
        target_var = "?answer"
        final_var, triples = self.process_join(parsed_expr, target_var)

        sparql_body = "\n  ".join([" ".join(t) + " ." for t in triples])
        sparql_query = f"""PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT DISTINCT {target_var} WHERE {{ 
  {sparql_body}
}}"""
        return sparql_query

WIKIDATA_SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"
WIKIDATA_API_ENDPOINT = "https://www.wikidata.org/w/api.php"

def execute_query_with_odbc(sparql_query):
    """Truy vấn Wikidata và trả về danh sách câu trả lời (bao gồm tất cả biến)"""
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/sparql-results+json"}
    response = requests.get(WIKIDATA_SPARQL_ENDPOINT, params={"query": sparql_query, "format": "json"}, headers=headers)

    if response.status_code == 200:
        results = response.json().get("results", {}).get("bindings", [])
        answers = []

        for result in results:
            for var in result:  # Duyệt qua tất cả các biến trả về
                value = result[var]["value"]
                answers.append(value)  # Chấp nhận tất cả giá trị, không chỉ thực thể Wikidata

        return answers  # Trả về toàn bộ danh sách kết quả

    return []

def convert_normed_to_s_expression(normed_expr): #, gold_entity_map, gold_relation_map
    """
    Chuyển đổi từ normed_sexpression sang s_expression.
    Sau khi trích xuất các nhãn quan hệ và thực thể từ normed_expr,
    ta lấy danh sách các mã ứng viên cho mỗi nhãn và tạo hoán vị giữa các cặp ứng viên đó.
    Kết quả trả về là một danh sách các s_expression khả dĩ.
    """
    normed_expr = fix_unbalanced_parentheses(normed_expr)
    # Trích xuất các nhãn quan hệ và thực thể từ biểu thức
    relations, entities = extract_entities_and_relations(normed_expr)
    # Tạo mapping từ token xuất hiện trong biểu thức sang danh sách các ứng viên mã.
    # Ví dụ: token_str = "[ author ]"
    s_expressions = []
    candidate_map = {}
    for rel in relations:
       
        token = f'[ {rel} ]'
        candidate = search_wikidata_relations(str(rel))

        # if not set(candidate)&set(gold_relation_map):
        #     return s_expressions 
        # Nếu candidate không phải danh sách, chuyển nó thành danh sách để tạo hoán vị
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
       
    for ent in entities:
        token = f'[ {ent} ]'
        candidate = search_wikidata_entities(str(ent))
        # if not set(candidate)&set(gold_entity_map):
        #     return s_expressions 
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
    
    # Nếu không có token nào cần thay thế, trả về biểu thức gốc

    if not candidate_map:
        return [normed_expr]
    
    # Lấy danh sách các token và danh sách các danh sách ứng viên tương ứng
    tokens = list(candidate_map.keys())
    candidate_lists = [candidate_map[token] for token in tokens]
    
    # Tạo tất cả các hoán vị ứng viên (Cartesian product)
    all_combinations = list(product(*candidate_lists))
    
    for comb in all_combinations:
        temp_expr = normed_expr
        # Với mỗi token, thay thế bằng ứng viên tương ứng theo hoán vị
        for token, replacement in zip(tokens, comb):
            temp_expr = temp_expr.replace(token, replacement)
        s_expressions.append(temp_expr)
    
    return s_expressions



In [40]:
import time

def run_result(query):
   
    query_result = []
    set_query = convert_normed_to_s_expression(query)

    for q in set_query:
        sparql = convert_s_expression_to_sparql(q)
        if sparql == "UNKNOWN":
            continue
        
        result = execute_query_with_odbc(sparql)

        # Nếu kết quả không rỗng, thêm vào danh sách
        if result:
            query_result.append(result)
  
    return query_result  # Trả về danh sách chứa danh sách con (chưa xử lý)

# Xử lý nhiều truy vấn
def main(question):
    start_time = time.time()
    
    queries = generate_query(question)
    all_results = []  # Chứa tất cả kết quả từ các truy vấn

    for i, query in enumerate(queries, 1):
        print(f"🔹 Query {i}: {query}")
        query_result = run_result(query)
        all_results.extend(query_result)  # Thêm danh sách kết quả vào all_results

    # Gộp danh sách con và loại bỏ rỗng
    final_results = sum(all_results, [])
    end_time = time.time()
    print(f"Execution time: {end_time - start_time} seconds")
    print("📌 Final Results:", final_results)  # Kết quả cuối cùng

# Chạy thử
question = "Việt Nam ở châu lục nào?"
main(question)


🔹 Query 1: ( JOIN [ lục địa ] [ Việt Nam ] )
🔹 Query 2: ( JOIN ( R [ nằm trong múi giờ ] ) [ Việt Nam ] )
🔹 Query 3: ( JOIN ( R [ nằm trong phạm vi của khu vực hành chính ] ) [ Việt Nam ] )
🔹 Query 4: ( JOIN ( R [ lục địa ] ) [ Việt Nam ] )
🔹 Query 5: 
Execution time: 7.850265026092529 seconds
📌 Final Results: ['http://www.wikidata.org/entity/Q63285961', 'http://www.wikidata.org/entity/Q6940', 'http://www.wikidata.org/entity/Q881', 'http://www.wikidata.org/entity/Q48', 'http://www.wikidata.org/entity/Q48', 'http://www.wikidata.org/entity/Q48']


In [10]:
# # Định nghĩa dữ liệu đầu vào cho API
# class InputData(BaseModel):
#     input_text: str

# class QueryInput(BaseModel):
#     question: str
    
# # Tạo endpoint để gọi model
# @app.post("/predict")
# def predict(data: InputData):
#     print("Received data:", data)
#     try:
#         # Tokenize input
#         input_ids = tokenizer.encode(data.input_text, return_tensors='pt').to(model.device)
#         print(input_ids)
#         # Generate output từ model
#         output = model.generate(input_ids, max_length=100)
#         output_text = tokenizer.decode(output[0], skip_special_tokens=True)
        
#         print("Generated output:", output_text)
#         return {"output_text": output_text}
#     except Exception as e:
#         print("Error during prediction:", str(e))
#         return {"error": str(e)}
        


# # API generate_query
# @app.post("/generate_query")
# def generate_query_api(data: QueryInput):
#     try:
#         print("Ques: "+data.question)
#         queries = generate_query(data.question)
#         print ("toiday")
       
#         return {"queries": queries}
#     except Exception as e:
#         return {"error": str(e)}

# # Sử dụng nest_asyncio để tránh lỗi asyncio trên Kaggle
# nest_asyncio.apply()

# # Thêm authtoken của bạn tại đây
# ngrok.set_auth_token("2tT6aa8W42iNmqXWfcwwAyzzokp_5Ko18vsKiFXBY6mz6sCwP")

# # Mở ngrok tunnel
# public_url = ngrok.connect(8000)
# print(f"Public URL: {public_url}")

# # Chạy ứng dụng bằng Uvicorn
# uvicorn.run(app, host='0.0.0.0', port=8000)